In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5

In [ ]:
from ICARUS.database import Database

# CHANGE THIS TO YOUR DATABASE FOLDER
database_folder = "E:\\Icarus\\Data"

# Load the database
DB = Database(database_folder)

In [ ]:
import numpy as np

from ICARUS.computation.solvers.AVL import (
    avl_dynamic_analysis_fd,
    process_avl_fd_res,
)
from ICARUS.environment import EARTH_ISA
from ICARUS.flight_dynamics.state import State

# Optimization

## Define Parameters

In [ ]:
plane = DB.get_vehicle("hermes")
plane.plot()

In [ ]:
from ICARUS.visualization.airplane import plot_airplane_polars

UINF = 20
solver2D = "Xfoil"
angles = np.linspace(-10, 10, 11)

state = State(name="Unstick", airplane=plane, environment=EARTH_ISA, u_freestream=UINF)

epsilons = {
    "u": 0.01 * UINF,
    "v": 0.01 * UINF,
    "w": 0.01 * UINF,
    "q": 0.005 * UINF,
    "p": 0.005 * UINF,
    "r": 0.005 * UINF,
    "theta": 0.01,
    "phi": 0.005 * UINF,
}

state.add_all_pertrubations("Central", epsilons)
avl_dynamic_analysis_fd(plane, state, solver2D, {"use_avl_control": True})
df = process_avl_fd_res(plane, state)
state.set_pertrubation_results(df)
state.stability_fd()
target_cl_cd = state.trim["CL/CD"]

plot_airplane_polars([plane.name])
state.plot_eigenvalues()

initial_longitudal_modes = state.state_space.longitudal.eigenvalues
initial_lateral_modes = state.state_space.lateral.eigenvalues

desired_lateral_modes = np.array(
    [-30.0 + 0.0j, -1.0 + 4.0j, -1.0 - 4.0j, -0.5 + 0.0j], dtype=complex
)

desired_longitudal_modes = np.array(
    [-1.0 - 4.0j, -1.0 + 4.0j, -0.1 - 0.4j, -0.1 + 0.4j], dtype=complex
)

## Define the optimization problem

In [ ]:
design_variables = [
    # Point Masses
    "payload_position_x",
    # Elevator
    "elevator_position_x",
    "elevator_root_chord",
    "elevator_tip_chord",
    # Rudder
    "rudder_position_x",
    "rudder_root_chord",
    "rudder_tip_chord",
]

bounds = {
    # Point Masses
    "payload_position_x": (0.0, 0.5),
    # Elevator
    "elevator_position_x": (0.4, 0.8),
    "elevator_root_chord": (0.05, 0.4),
    "elevator_tip_chord": (0.01, 0.1),
    "elevator_pitch": (-3, 3),
    # Rudder
    "rudder_position_x": (0.4, 0.8),
    "rudder_root_chord": (0.05, 0.4),
    "rudder_tip_chord": (0.05, 0.2),
}

desing_constants = {
    "state": state,
}

## Define Objective Function

In [ ]:
from ICARUS.flight_dynamics.trim import TrimNotPossible, TrimOutsidePolars
from ICARUS.vehicle.airplane import Airplane


def obj_fun(
    plane: Airplane,
    state: State,
):
    state.update_plane(plane)
    state.add_all_pertrubations("Central")
    try:
        avl_dynamic_analysis_fd(plane, state, "Xfoil")
    except (TrimNotPossible, TrimOutsidePolars):
        return np.inf

    df = process_avl_fd_res(plane, state)
    state.set_pertrubation_results(df)
    state.stability_fd()

    # longitudal_eigs: ComplexArray = unstick.state_space.longitudal.eigenvalues
    # lateral_eigs: ComplexArray = unstick.state_space.lateral.eigenvalues

    # Get the eigenvalues and the damping ratios for the longitudinal modes
    longitudal_modes = state.state_space.longitudal.eigenvalues
    lateral_modes = state.state_space.lateral.eigenvalues

    # Compute the fitness function
    fitness = 0.0
    for i in range(4):
        fitness += (
            np.linalg.norm(longitudal_modes[i] - desired_longitudal_modes[i]) ** 2
        )
        fitness += np.linalg.norm(lateral_modes[i] - desired_longitudal_modes[i]) ** 2

    return float(fitness)


def constraint_cl_over_cd(state: State):
    return state.trim["CL/CD"] - target_cl_cd + 1


def constraint_cm_a_pos(state: State):
    if "Cm0" not in state.trim.keys():
        return np.inf
    return state.trim["Cm0"]


def constraint_rudder_position(
    plane: Airplane,
):
    # Set the position of the elevator and rudder
    eleve_position_x = plane.__getattribute__("elevator_position_x")
    plane.__setattr__("rudder_position_x", eleve_position_x)

## Run Optimizer

In [ ]:
from ICARUS.optimization.callbacks.design_variables_vis import DesignVariableVisualizer
from ICARUS.optimization.callbacks.eigenvalue_opt_vis import (
    EigenvalueOptimizationVisualizer,
)
from ICARUS.optimization.callbacks.optimization_progress_vis import OptimizationProgress
from ICARUS.optimization.callbacks.plane_geometry_vis import PlaneGeometryVisualization
from ICARUS.optimization.callbacks.plane_polars_vis import (
    AirplanePolarOptimizationVisualizer,
)
from ICARUS.optimization.optimizers.airplane.airplane_dynamics_optimizer import (
    Airplane_Dynamics_Optimizer,
)

# Design variable callback
design_variable_callback = DesignVariableVisualizer(
    plane=plane,
    design_variables=design_variables,
    bounds=bounds,
)

# Progress callback
progress_callback = OptimizationProgress()

# Plane geometry callback
plane_geometry_callback = PlaneGeometryVisualization(plane=plane)

# Eigenvalue callback
eigenvalue_callback = EigenvalueOptimizationVisualizer(
    goal_longitudal=desired_longitudal_modes,
    goal_lateral=desired_lateral_modes,
    initial_longitudal=initial_longitudal_modes,
    initial_lateral=initial_lateral_modes,
)

polar_callback = AirplanePolarOptimizationVisualizer(
    initial_state=state,
)

optimizer = Airplane_Dynamics_Optimizer(
    design_variables=design_variables,
    design_constants=desing_constants,
    plane=plane,
    state=state,
    f=obj_fun,
    non_linear_constraints=[
        {"type": "ineq", "fun": constraint_cl_over_cd, "lb": -1.0, "ub": 1.0},
        {"type": "ineq", "fun": constraint_cm_a_pos, "lb": 0.0, "ub": np.inf},
    ],
    bounds=bounds,
    jac=None,
    callback_list=[
        eigenvalue_callback,
        design_variable_callback,
        progress_callback,
        polar_callback,
        # plane_geometry_callback
    ],
)

In [ ]:
res = optimizer("Nelder-Mead", options={"maxiter": 1000})

# Get Results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

optimizer.current_obj.visualize(fig, ax)  # type: ignore
optimizer.current_obj.visualize(fig, ax)  # type: ignore

In [ ]:
plane_1: Airplane = optimizer.initial_obj
plane_2: Airplane = optimizer.current_obj

unstick_1 = State(
    name="Unstick", airplane=plane_1, environment=EARTH_ISA, u_freestream=UINF
)
unstick_1.add_all_pertrubations("Central")
avl_dynamic_analysis_fd(plane_1, unstick_1, solver2D="Xfoil")
df = process_avl_fd_res(plane_1, unstick_1)
unstick_1.set_pertrubation_results(df)
unstick_1.stability_fd()

In [ ]:
unstick_2 = State(
    name="Unstick", airplane=plane_2, environment=EARTH_ISA, u_freestream=UINF
)
unstick_2.add_all_pertrubations("Central")
avl_dynamic_analysis_fd(plane_2, unstick_2, solver2D="Xfoil")
df = process_avl_fd_res(plane_2, unstick_2)
unstick_2.set_pertrubation_results(df)
unstick_2.stability_fd()